# Extract burst number for SNAP TOPS Split

In [1]:
import urllib
import json

from IPython.display import JSON

## 1) Read the metadata for the burst of interest

Generate the query for a sigle bursts with the following inputs:

In [2]:
AcquisitionDate = '2024-08-14'
PolarisationChannels = 'VV'
x = 10.756
y = 46.747

In [3]:
https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ContentDate/Start ge {AcquisitionDate}T00:00:00.000Z and ContentDate/Start le {AcquisitionDate}T23:59:59.000Z and "
    f"PolarisationChannels eq '{PolarisationChannels}' and "
    f"OData.CSC.Intersects(area=geography'SRID=4326;POINT({x} {y})')"
) + "&$top=1000"
print(https_request)

https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=ContentDate/Start%20ge%202024-08-14T00%3A00%3A00.000Z%20and%20ContentDate/Start%20le%202024-08-14T23%3A59%3A59.000Z%20and%20PolarisationChannels%20eq%20%27VV%27%20and%20OData.CSC.Intersects%28area%3Dgeography%27SRID%3D4326%3BPOINT%2810.756%2046.747%29%27%29&$top=1000


Read the json and extract `ProductName`, `BurstId` and `SwathIdentifier`

In [4]:
with urllib.request.urlopen(https_request) as response:
    content = response.read().decode()
bursts = json.loads(content)
JSON(bursts)

<IPython.core.display.JSON object>

In [5]:
ProductName = bursts['value'][0]['ParentProductName']
BurstId = bursts['value'][0]['BurstId']
SwathIdentifier = bursts['value'][0]['SwathIdentifier']

print(f'ProductName: {ProductName}\nBurstId: {BurstId}\nSwathIdentifier: {SwathIdentifier}')

ProductName: S1A_IW_SLC__1SDV_20240814T171540_20240814T171607_055212_06BAD3_C31B.SAFE
BurstId: 30345
SwathIdentifier: IW3


## 2) Extract the burst number for the SNAP TOPS Split function

List all the burst ids in the subswath product which contains the subswath of interest

In [6]:
https_request = "https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ParentProductName eq '{ProductName}' and "
    f"SwathIdentifier eq '{SwathIdentifier}' and "
    f"PolarisationChannels eq '{PolarisationChannels}'"
) + "&$orderby=ContentDate/Start&$top=1000"
print(https_request)

https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=ParentProductName%20eq%20%27S1A_IW_SLC__1SDV_20240814T171540_20240814T171607_055212_06BAD3_C31B.SAFE%27%20and%20SwathIdentifier%20eq%20%27IW3%27%20and%20PolarisationChannels%20eq%20%27VV%27&$orderby=ContentDate/Start&$top=1000


In [7]:
with urllib.request.urlopen(https_request) as response:
    content = response.read().decode()
bursts_in_subswath = json.loads(content)
JSON(bursts_in_subswath)

<IPython.core.display.JSON object>

In [8]:
[(b['BurstId'], b['BeginningDateTime']) for b in bursts_in_subswath['value']]

[(30342, '2024-08-14T17:15:41.995309Z'),
 (30343, '2024-08-14T17:15:44.753586Z'),
 (30344, '2024-08-14T17:15:47.511863Z'),
 (30345, '2024-08-14T17:15:50.270140Z'),
 (30346, '2024-08-14T17:15:53.028417Z'),
 (30347, '2024-08-14T17:15:55.786694Z'),
 (30348, '2024-08-14T17:15:58.544971Z'),
 (30349, '2024-08-14T17:16:01.303248Z'),
 (30350, '2024-08-14T17:16:04.061525Z')]

In [12]:
burstId_list = [b['BurstId'] for b in bursts_in_subswath['value']]
tops_split_number = BurstId - min(burstId_list) + 1
print(f'Subswath: {SwathIdentifier}, Burst number: {tops_split_number}')

Subswath: IW3, Burst number: 4


## 3) Get the data from the S3 bucket


In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

if PolarisationChannels=="VV":
    exclude_pol='vh'
elif PolarisationChannels=="VH":
    exclude_pol='vv'

s3_endpoint = "eodata.dataspace.copernicus.eu"
subswath_id = SwathIdentifier.lower()
in_path = bursts["value"][0]["S3Path"].split(".SAFE")[0] + ".SAFE"
output_path = bursts["value"][0]["S3Path"].split(".SAFE")[0].split("/")[-1] + ".SAFE/"

In [ ]:
os.system(f"s5cmd --endpoint-url \"https://{s3_endpoint}\" cp --include \"*{subswath_id}*\" --exclude \"*{exclude_pol}*\" --include \"manifest.safe\" \"s3:/\"{in_path}\"/*\" {output_path}/")